In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("./properatti.csv", index_col=0)
data.shape

(121220, 25)

### Vistas iniciales del dataset de Properatti

In [4]:
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

In [5]:
data.tail(10)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
121210,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,NaN,...,120.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cj8o_venta_casa_m...,Dos viviendas en block en excelente ubicación ...,Casa - Martinez,https://thumbs4.properati.com/7/3-sXSLx8KPDZzR...
121211,sell,apartment,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,NaN,...,205.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cj9y_venta_depart...,Excelente departamento de categoría con vistas...,Departamento - Acassuso,https://thumbs4.properati.com/0/pvevdNsjE5Kf4r...
121212,sell,house,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,NaN,...,420.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cj9z_venta_casa_a...,ESPECTACULAR CASA ESTILO RACIONALISTA EN CALLE...,Casa - Acassuso,https://thumbs4.properati.com/7/DiXqpull13ebTp...
121213,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,NaN,...,600.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cja0_venta_casa_m...,EXCELENTE casa en Martinez. Inmejorable ubicai...,Casa - Martinez,https://thumbs4.properati.com/8/v0Il0v9Key--5Z...
121214,sell,store,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,NaN,NaN,NaN,...,123.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cja1_venta_local_...,***VENTA CON RENTA***Local en EDIFICIO lomas ...,Local - San Isidro,https://thumbs4.properati.com/7/R-j0JiSePtAriH...
121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,-58.475596,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...
121219,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,NaN,...,77.0,1675.324675,1675.324675,NaN,NaN,3500.0,http://www.properati.com.ar/1cjaa_venta_depart...,Departamento de 77 m2 apto profesional. Edific...,Departamento apto profesional en el Centro,https://thumbs4.properati.com/8/mNEsnQWqB7joBF...


Analizamos por lat-lon para si había duplicados considerando que las mismas coordenadas son las misma propiedad.

No es posible determinar si es la misma propiedad porque lat-lon se refiere muchas veces a la ubicación aproximada. 



In [6]:
## ver si existe algun dato duplicado
data.duplicated().any()

False

In [7]:
data['lat-lon'].duplicated().any()

True

In [8]:
## ver datos duplicados
data_copy = data.copy()
data_copy.dropna(subset=['lat-lon'], inplace=True)
data_copy_group = data_copy.groupby('lat-lon').count()
data_copy_group[data_copy_group['operation'] > 1].sort_values(by='operation', ascending=False)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat,lon,price,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
lat-lon,,,,,,,,,,,,,,,,,,,,,
"-34.4026444,-58.6684776",312,312,312,312,312,312,0,312,312,309,...,312,306,309,0,231,0,312,312,312,312
"-34.40014,-58.64985",271,271,271,271,271,271,0,271,271,270,...,270,264,269,2,205,1,271,271,271,271
"-34.6428675967,-58.4376599743",259,259,259,259,259,259,259,259,259,6,...,4,6,3,0,0,0,259,259,259,259
"-34.425087,-58.5796585",224,224,223,224,224,224,107,224,224,213,...,220,199,209,0,81,0,224,224,224,224
"-34.5895459,-58.3973636",156,156,156,156,156,156,156,156,156,148,...,156,148,148,2,144,2,156,156,156,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"-34.5841733,-58.4008772",2,2,2,2,2,2,2,2,2,2,...,2,2,2,0,0,2,2,2,2,2
"-34.58412,-58.453668",2,2,2,2,2,2,2,2,2,2,...,2,0,2,2,2,0,2,2,2,2
"-34.5840538,-58.4346066",2,2,2,2,2,2,0,2,2,2,...,2,2,2,0,2,0,2,2,2,2


In [9]:
data_copy.shape

(69670, 25)

In [10]:
data_copy[data_copy['lat-lon'] == '-34.4026444,-58.6684776']

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
1284,sell,apartment,Barrio Los Alisos,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,52.0,3203.389831,3634.615385,NaN,2.0,NaN,http://www.properati.com.ar/15hsc_venta_depart...,Departamento de 2 ambientes con vista al lago ...,PORTEZUELO | Marinas del Portezuelo - 2 ambien...,https://thumbs4.properati.com/9/a49C3o5BkTdafI...
1287,sell,apartment,Barrio Los Alisos,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,49.0,2894.736842,3367.346939,NaN,2.0,NaN,http://www.properati.com.ar/15hsi_venta_depart...,"Departamento de 2 ambientes muy luminoso, bar...",PORTEZUELO | Marinas del Portezuelo - Departam...,https://thumbs4.properati.com/9/7f4Vk-cijSd_Vd...
1306,sell,apartment,Barrio Los Alisos,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,54.0,2463.768116,3148.148148,NaN,2.0,NaN,http://www.properati.com.ar/15htl_venta_depart...,Unidad de 2 ambientes muy luminosa. Dormitorio...,PORTEZUELO | Marinas del Portezuelo - Albatros...,https://thumbs4.properati.com/3/0ivvcFWZzuAU3t...
4648,sell,apartment,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,76.0,2171.052632,2171.052632,NaN,2.0,NaN,http://www.properati.com.ar/15nbg_venta_depart...,"Departamento a estrenar en edificio Delta 8 , ...","Departamento en venta en Edificio Delta 8, Nor...",https://thumbs4.properati.com/5/tWSqJjKwu1i53r...
9502,sell,house,Barrio Los Alisos,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,295.0,2660.818713,3084.745763,NaN,NaN,NaN,http://www.properati.com.ar/164jj_venta_casa_n...,"Excelente casa a estrenar, entrega Julio 2017....",Hermosa Casa a estrenar en Lagos del Golf - No...,https://thumbs4.properati.com/8/JqL-_BgzwTcb9h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113559,sell,apartment,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,58.0,1623.931624,3275.862069,NaN,2.0,NaN,http://www.properati.com.ar/1c8ie_venta_depart...,"""Departamento MUY luminoso de 2 ambientes y 1/...",Departamento 2 1/2 ambientes con vista al rio ...,https://thumbs4.properati.com/6/i7DP23BhojHDVu...
114089,sell,apartment,BarrioPortezuelo,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,83.0,2231.404959,3253.012048,NaN,3.0,NaN,http://www.properati.com.ar/1c9dc_venta_depart...,Living comedor y escritorio en L con salida al...,3 ambientes en esquina en venta a estrenar con...,https://thumbs4.properati.com/1/L4o5Ic0mJU-WgZ...
114409,sell,apartment,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,68.0,1886.363636,3661.764706,NaN,3.0,NaN,http://www.properati.com.ar/1c9ue_venta_depart...,Muy linda planta baja con jardín y amplia gale...,Venta amplio departamento en PB,https://thumbs4.properati.com/3/xM5r5TAlaez-FQ...
114425,sell,house,Barrio La Alameda,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026444,-58.6684776",-34.402644,-58.668478,...,177.0,937.500000,2542.372881,NaN,NaN,NaN,http://www.properati.com.ar/1c9vc_venta_casa_b...,Espectacular casa construida en dos plantas a ...,Espectacular casa en el barrio

In [11]:
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

#### Análisis de datos faltantes

In [12]:
missing_data = data.isna().sum(axis=0)
missing_data_df = pd.DataFrame(missing_data, columns=['count'])
missing_data_df['perc'] = (missing_data_df / data.shape[0]).round(2)*100
missing_data_df.sort_values(ascending=False, by='count')

,count,perc
floor,113321,93.0
expenses,106958,88.0
rooms,73830,61.0
price_usd_per_m2,52603,43.0
lat-lon,51550,43.0
lat,51550,43.0
lon,51550,43.0
surface_total_in_m2,39328,32.0
price_per_m2,33562,28.0
currency,20411,17.0


#### Dispersión de datos

In [13]:
data_dispersion = data.apply(lambda x: x.unique().size)
data_dispersion_df = pd.DataFrame(data_dispersion, columns=['count'])
data_dispersion_df["perc"] = (data_dispersion / data.shape[0]).round(2)*100
data_dispersion_df.sort_values(ascending=True, by='count')

,count,perc
operation,1,0.0
country_name,1,0.0
property_type,4,0.0
currency,5,0.0
state_name,28,0.0
rooms,32,0.0
floor,183,0.0
geonames_id,647,1.0
expenses,983,1.0
surface_covered_in_m2,996,1.0


In [14]:
for col in data.columns:
    if(data[col].nunique() < 100):
        print(col)
        print(data[col].unique())
        print()

operation
['sell']

property_type
['PH' 'apartment' 'house' 'store']

country_name
['Argentina']

state_name
['Capital Federal' 'Bs.As. G.B.A. Zona Sur' 'Buenos Aires Costa Atlántica'
 'Entre Ríos' 'Bs.As. G.B.A. Zona Norte' 'Santa Fe' 'Córdoba'
 'Bs.As. G.B.A. Zona Oeste' 'Misiones' 'Buenos Aires Interior' 'Salta'
 'Neuquén' 'Río Negro' 'San Luis' 'Mendoza' 'Corrientes' 'Chubut'
 'Tucumán' 'La Pampa' 'Chaco' 'San Juan' 'Santa Cruz' 'Tierra Del Fuego'
 'Catamarca' 'Santiago Del Estero' 'Jujuy' 'La Rioja' 'Formosa']

currency
['USD' nan 'ARS' 'PEN' 'UYU']

rooms
[nan  1.  4.  3.  2.  6.  5. 10.  7.  9.  8. 17. 22. 15. 12. 11. 14. 16.
 20. 13. 25. 19. 30. 18. 32. 24. 31. 21. 29. 27. 23. 28.]

